In [110]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
import sys as sys
%matplotlib inline

$\textbf{Making the Model}$

In [5]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [6]:
x = mobile.layers[-5].output
x = tf.keras.layers.Reshape(target_shape=(1024,))(x)
output = Dense(units = 120, activation='softmax')(x)

In [7]:
model = Model(inputs = mobile.input, outputs = output)

In [8]:
for layer in model.layers[:-22]:
    layer.trainable = False 

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [11]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics =['accuracy'])

$\textbf{Image Preperation}$

In [80]:
os.getcwd()

'/Users/bivanshrestha/Desktop/Notebooks/Mobile Net Practice/Data'

In [70]:
count = 1
dog_dict = {}
for file in os.listdir('Images'):
    if file != '.DS_Store':
        dog_dict[count] = file
        count +=1

1: n02097658-silky_terrier
2: n02092002-Scottish_deerhound
3: n02099849-Chesapeake_Bay_retriever
4: n02091244-Ibizan_hound
5: n02095314-wire-haired_fox_terrier
6: n02091831-Saluki
7: n02102318-cocker_spaniel
8: n02104365-schipperke
9: n02090622-borzoi
10: n02113023-Pembroke
11: n02105505-komondor
12: n02093256-Staffordshire_bullterrier
13: n02113799-standard_poodle
14: n02109961-Eskimo_dog
15: n02089973-English_foxhound
16: n02099601-golden_retriever
17: n02095889-Sealyham_terrier
18: n02085782-Japanese_spaniel
19: n02097047-miniature_schnauzer
20: n02110063-malamute
21: n02105162-malinois
22: n02086079-Pekinese
23: n02097130-giant_schnauzer
24: n02113978-Mexican_hairless
25: n02107142-Doberman
26: n02097209-standard_schnauzer
27: n02115913-dhole
28: n02106662-German_shepherd
29: n02106382-Bouvier_des_Flandres
30: n02110185-Siberian_husky
31: n02094258-Norwich_terrier
32: n02093991-Irish_terrier
33: n02094114-Norfolk_terrier
34: n02109525-Saint_Bernard
35: n02093754-Border_terrier
36: n02105251-briard
37: n02108551-Tibetan_mastiff
38: n02108422-bull_mastiff
39: n02085936-Maltese_dog
40: n02093859-Kerry_blue_terrier
41: n02104029-kuvasz
42: n02107574-Greater_Swiss_Mountain_dog
43: n02095570-Lakeland_terrier
44: n02086646-Blenheim_spaniel
45: n02088238-basset
46: n02098286-West_Highland_white_terrier
47: n02085620-Chihuahua
48: n02106166-Border_collie
49: n02090379-redbone
50: n02090721-Irish_wolfhound
51: n02088632-bluetick
52: n02113712-miniature_poodle
53: n02113186-Cardigan
54: n02108000-EntleBucher
55: n02091467-Norwegian_elkhound
56: n02100236-German_short-haired_pointer
57: n02107683-Bernese_mountain_dog
58: n02086910-papillon
59: n02097474-Tibetan_terrier
60: n02101006-Gordon_setter
61: n02093428-American_Staffordshire_terrier
62: n02100583-vizsla
63: n02105412-kelpie
64: n02092339-Weimaraner
65: n02107312-miniature_pinscher
66: n02108089-boxer
67: n02112137-chow
68: n02105641-Old_English_sheepdog
69: n02110958-pug
70: n02087394-Rhodesian_ridgeback
71: n02097298-Scotch_terrier
72: n02086240-Shih-Tzu
73: n02110627-affenpinscher
74: n02091134-whippet
75: n02102480-Sussex_spaniel
76: n02091635-otterhound
77: n02099267-flat-coated_retriever
78: n02100735-English_setter
79: n02091032-Italian_greyhound
80: n02099712-Labrador_retriever
81: n02106030-collie
82: n02096177-cairn
83: n02106550-Rottweiler
84: n02096294-Australian_terrier
85: n02087046-toy_terrier
86: n02105855-Shetland_sheepdog
87: n02116738-African_hunting_dog
88: n02111277-Newfoundland
89: n02089867-Walker_hound
90: n02098413-Lhasa
91: n02088364-beagle
92: n02111889-Samoyed
93: n02109047-Great_Dane
94: n02096051-Airedale
95: n02088466-bloodhound
96: n02100877-Irish_setter
97: n02112350-keeshond
98: n02096437-Dandie_Dinmont
99: n02110806-basenji
100: n02093647-Bedlington_terrier
101: n02107908-Appenzeller
102: n02101556-clumber
103: n02113624-toy_poodle
104: n02111500-Great_Pyrenees
105: n02102040-English_springer
106: n02088094-Afghan_hound
107: n02101388-Brittany_spaniel
108: n02102177-Welsh_springer_spaniel
109: n02096585-Boston_bull
110: n02115641-dingo
111: n02098105-soft-coated_wheaten_terrier
112: n02099429-curly-coated_retriever
113: n02108915-French_bulldog
114: n02102973-Irish_water_spaniel
115: n02112018-Pomeranian
116: n02112706-Brabancon_griffon
117: n02094433-Yorkshire_terrier
118: n02105056-groenendael
119: n02111129-Leonberg
120: n02089078-black-and-tan_coonhound

In [81]:
os.chdir('/Users/bivanshrestha/Desktop/Notebooks/Dog Classification/Dog Breed Dataset')


for i in range(1, 121):
    os.mkdir(f'train/{i}')
    os.mkdir(f'valid/{i}')
    os.mkdir(f'test/{i}')
        
    train_samples = random.sample(os.listdir(f'Images/{dog_dict[i]}'), 110)                    
    for t in train_samples:
        shutil.move(f'Images/{dog_dict[i]}/{t}', f'train/{i}')
    
    valid_samples = random.sample(os.listdir(f'Images/{dog_dict[i]}'), 25)
    for v in valid_samples:
        shutil.move(f'Images/{dog_dict[i]}/{v}', f'valid/{i}')                
                    
    test_samples = random.sample(os.listdir(f'Images/{dog_dict[i]}'), 15)
    for test in test_samples:
        shutil.move(f'Images/{dog_dict[i]}/{test}', f'test/{i}')
    

In [47]:
os.getcwd()

'/Users/bivanshrestha/Desktop/Notebooks/Dog Classification/Dog Breed Dataset'

In [82]:
train_path = '/Users/bivanshrestha/Desktop/Notebooks/Dog Classification/Dog Breed Dataset/Train'
valid_path = '/Users/bivanshrestha/Desktop/Notebooks/Dog Classification/Dog Breed Dataset/Valid'
test_path = '/Users/bivanshrestha/Desktop/Notebooks/Dog Classification/Dog Breed Dataset/Test'

In [95]:
train_batches = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory = train_path, target_size = (224, 224), batch_size = 10)

valid_batches = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory = valid_path, target_size = (224, 224), batch_size = 10)

test_batches = ImageDataGenerator(preprocessing_function = tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory = test_path, target_size = (224, 224), batch_size = 10, shuffle = False)

Found 13200 images belonging to 120 classes.
Found 3000 images belonging to 120 classes.
Found 1800 images belonging to 120 classes.


$\textbf{Train the Model}$

In [84]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics =['accuracy'])

In [85]:
model.fit(x=train_batches,validation_data=valid_batches, epochs=10, verbose=2)

Epoch 1/10


2023-06-21 15:21:00.915390: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1320/1320 - 294s - loss: 2.3516 - accuracy: 0.4405 - val_loss: 1.2044 - val_accuracy: 0.6667 - 294s/epoch - 223ms/step
Epoch 2/10
1320/1320 - 292s - loss: 1.0266 - accuracy: 0.7333 - val_loss: 0.9786 - val_accuracy: 0.7117 - 292s/epoch - 221ms/step
Epoch 3/10
1320/1320 - 285s - loss: 0.6831 - accuracy: 0.8339 - val_loss: 0.9270 - val_accuracy: 0.7213 - 285s/epoch - 216ms/step
Epoch 4/10
1320/1320 - 281s - loss: 0.4544 - accuracy: 0.9043 - val_loss: 0.8945 - val_accuracy: 0.7347 - 281s/epoch - 213ms/step
Epoch 5/10
1320/1320 - 280s - loss: 0.3073 - accuracy: 0.9433 - val_loss: 0.8970 - val_accuracy: 0.7357 - 280s/epoch - 212ms/step
Epoch 6/10
1320/1320 - 282s - loss: 0.2114 - accuracy: 0.9693 - val_loss: 0.8744 - val_accuracy: 0.7397 - 282s/epoch - 214ms/step
Epoch 7/10
1320/1320 - 341s - loss: 0.1512 - accuracy: 0.9813 - val_loss: 0.8635 - val_accuracy: 0.7397 - 341s/epoch - 258ms/step
Epoch 8/10
1320/1320 - 385s - loss: 0.1089 - accuracy: 0.9881 - val_loss: 0.9044 - val_accuracy: 0.73

$\textbf{Predicting Dog Breed}$

In [96]:
test_labels = test_batches.classes

In [97]:
predictions = model.predict (x=test_batches, verbose = 0)

In [98]:
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))

In [99]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [101]:
test_batches.class_indices

{'1': 0,
 '10': 1,
 '100': 2,
 '101': 3,
 '102': 4,
 '103': 5,
 '104': 6,
 '105': 7,
 '106': 8,
 '107': 9,
 '108': 10,
 '109': 11,
 '11': 12,
 '110': 13,
 '111': 14,
 '112': 15,
 '113': 16,
 '114': 17,
 '115': 18,
 '116': 19,
 '117': 20,
 '118': 21,
 '119': 22,
 '12': 23,
 '120': 24,
 '13': 25,
 '14': 26,
 '15': 27,
 '16': 28,
 '17': 29,
 '18': 30,
 '19': 31,
 '2': 32,
 '20': 33,
 '21': 34,
 '22': 35,
 '23': 36,
 '24': 37,
 '25': 38,
 '26': 39,
 '27': 40,
 '28': 41,
 '29': 42,
 '3': 43,
 '30': 44,
 '31': 45,
 '32': 46,
 '33': 47,
 '34': 48,
 '35': 49,
 '36': 50,
 '37': 51,
 '38': 52,
 '39': 53,
 '4': 54,
 '40': 55,
 '41': 56,
 '42': 57,
 '43': 58,
 '44': 59,
 '45': 60,
 '46': 61,
 '47': 62,
 '48': 63,
 '49': 64,
 '5': 65,
 '50': 66,
 '51': 67,
 '52': 68,
 '53': 69,
 '54': 70,
 '55': 71,
 '56': 72,
 '57': 73,
 '58': 74,
 '59': 75,
 '6': 76,
 '60': 77,
 '61': 78,
 '62': 79,
 '63': 80,
 '64': 81,
 '65': 82,
 '66': 83,
 '67': 84,
 '68': 85,
 '69': 86,
 '7': 87,
 '70': 88,
 '71': 89,
 '72':

In [102]:
cm_plot_labels = []
for i in range(0, 120):
    cm_plot_labels.append(i)

In [118]:
np.set_printoptions(threshold=1000)

In [121]:
#So we have a lot of classes, so a confusion matrix isn't really the best way to represent the predicted data
#Will have to find a better way to represent all of this data
cm

array([[11,  0,  0, ...,  0,  0,  0],
       [ 0, 10,  0, ...,  0,  0,  0],
       [ 0,  0, 15, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 15,  0,  0],
       [ 0,  0,  0, ...,  0, 13,  0],
       [ 0,  0,  0, ...,  0,  0, 11]])

$\textbf{Data augmentation for more training images}$

In [ ]:
#Here we will flip all the images across the vertical axis to double our training datasize
import Augmentor 
